In [9]:
import pandas as pd 
import numpy as np 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
df = pd.read_csv(r"/content/cea-salespersons-property-transaction-records-residential.csv")
df.head()

,salesperson_name,transaction_date,salesperson_reg_num,property_type,sale_type,represented,town,district,general_location
0,"TAN MIN ER, JOANNA (CHEN MINER, JOANNA) (JOANN...",Jan-17,R041140C,HDB,RESALE,BUYER,ANG MO KIO,-,-
1,TEO KIM LENG (RON TEO),Jan-17,R048433H,HDB,RESALE,SELLER,ANG MO KIO,-,-
2,WONG VINCENT (WONG VINCENT),Jan-17,R027375B,HDB,RESALE,BUYER,ANG MO KIO,-,-
3,TAN BENG LOO (CHEN MINGRU) (YVONNE TAN),Jan-17,R006853I,HDB,RESALE,SELLER,ANG MO KIO,-,-
4,TAN MON WAN (EDDIE TAN),Jan-17,R011153A,HDB,RESALE,BUYER,ANG MO KIO,-,-


In [10]:
cleaned_df = df.query('property_type =="HDB" & sale_type == "RESALE" & represented == "BUYER"')
cleaned_df = cleaned_df.drop(['salesperson_reg_num', 'property_type','sale_type','represented','district','general_location'], axis = 1)
cleaned_df.head() 

,salesperson_name,transaction_date,town
0,"TAN MIN ER, JOANNA (CHEN MINER, JOANNA) (JOANN...",Jan-17,ANG MO KIO
2,WONG VINCENT (WONG VINCENT),Jan-17,ANG MO KIO
4,TAN MON WAN (EDDIE TAN),Jan-17,ANG MO KIO
6,TEE BEE KHIM (BRENDA TEE),Jan-17,ANG MO KIO
8,CHEN REN JIE (ELSIE),Jan-17,ANG MO KIO


In [11]:
yishun_sellers = cleaned_df.query('town == "YISHUN"') 
semb_sellers = cleaned_df.query('town == "SEMBAWANG"') 
yishun_arr =yishun_sellers['salesperson_name'].to_numpy()
semb_arr = semb_sellers['salesperson_name'].to_numpy()
yishun_arr = np.unique(yishun_arr) 
semb_arr = np.unique(semb_arr)
common_arr = np.intersect1d(yishun_arr,semb_arr)


In [12]:
result = cleaned_df[cleaned_df.salesperson_name.isin(common_arr)]
result

,salesperson_name,transaction_date,town
21,ELAINE CHIA WEE LING,Jan-17,ANG MO KIO
24,LIM CHEE KEONG (LIN ZHIQIANG) (LIM CHEE KEONG),Jan-17,ANG MO KIO
33,TAN SIEW ENG,Jan-17,ANG MO KIO
42,MOHAMAD ZAIDIL BIN MOHAMAD FADILLAH (ZAI),Jan-17,ANG MO KIO
74,CHAN JIEMIN,Jan-17,ANG MO KIO
...,...,...,...
317528,LIM HONG PENG (LIM HONG PENG),Sep-21,YISHUN
317541,MOHAMAD ZAIDIL BIN MOHAMAD FADILLAH (ZAI),Sep-21,YISHUN
317542,CHAI CHEE KONG,Sep-21,YISHUN
317557,"LIM KUN RONG, STEWART (STEWART LIM)",Sep-21,YISHUN


In [13]:
result = result.groupby(['salesperson_name', 'town']).size().reset_index(name= 'count') 
#result = result.fillna(0) 
result.loc[result['count'] > 0, 'count'] = 1
result

,salesperson_name,town,count
0,ABDUL SHAMIM SHAH S/O SHAKUR SHAH,SEMBAWANG,1
1,ABDUL SHAMIM SHAH S/O SHAKUR SHAH,YISHUN,1
2,AHMAD BIN IBRAHIM,ANG MO KIO,1
3,AHMAD BIN IBRAHIM,BEDOK,1
4,AHMAD BIN IBRAHIM,BUKIT BATOK,1
...,...,...,...
4641,ng guan wee rony (Mr Boleh Rony Ng),SERANGOON,1
4642,ng guan wee rony (Mr Boleh Rony Ng),TAMPINES,1
4643,ng guan wee rony (Mr Boleh Rony Ng),TOA PAYOH,1
4644,ng guan wee rony (Mr Boleh Rony Ng),WOODLANDS,1


In [14]:
result = result.pivot(index = 'salesperson_name', columns = 'town', values ='count')
result = result.fillna(0) 
result

town,ANG MO KIO,BEDOK,BISHAN,BUKIT BATOK,BUKIT MERAH,BUKIT PANJANG,BUKIT TIMAH,CENTRAL AREA,CHOA CHU KANG,CLEMENTI,GEYLANG,HOUGANG,JURONG EAST,JURONG WEST,KALLANG/WHAMPOA,MARINE PARADE,PASIR RIS,PUNGGOL,QUEENSTOWN,SEMBAWANG,SENGKANG,SERANGOON,TAMPINES,TOA PAYOH,WOODLANDS,YISHUN
salesperson_name,,,,,,,,,,,,,,,,,,,,,,,,,,
ABDUL SHAMIM SHAH S/O SHAKUR SHAH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
AHMAD BIN IBRAHIM,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
AIDA NORZEHAN BINTI AB TALIB,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
ALIFMIRZAN BIN KAMARUDIN (YA ALIF),1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
ALVIN LAW CHUNG SIONG (LIU JUNXIONG) (ALVIN),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZHAN JUANYONG (ALEX ZHAN),1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
ZHANG WEILU (WILLIAM ZHANG),1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
ZHENG YANHUI (ALEX),0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


In [15]:

# Building the model
frq_items = apriori(result, min_support = 0.06, use_colnames = True)
 
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

In [16]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1128047,"(TOA PAYOH, WOODLANDS, BUKIT PANJANG, HOUGANG,...","(BEDOK, CHOA CHU KANG)",0.061728,0.257202,0.061728,1.000000,3.888,0.045852,inf
2444251,"(TOA PAYOH, WOODLANDS, BUKIT PANJANG, HOUGANG,...","(BEDOK, CHOA CHU KANG)",0.061728,0.257202,0.061728,1.000000,3.888,0.045852,inf
2444296,"(TOA PAYOH, WOODLANDS, BUKIT PANJANG, HOUGANG,...","(SEMBAWANG, BEDOK, CHOA CHU KANG)",0.061728,0.257202,0.061728,1.000000,3.888,0.045852,inf
2446029,"(TOA PAYOH, WOODLANDS, YISHUN, BUKIT PANJANG, ...","(BEDOK, CHOA CHU KANG)",0.061728,0.257202,0.061728,1.000000,3.888,0.045852,inf
2446076,"(TOA PAYOH, WOODLANDS, BUKIT PANJANG, HOUGANG,...","(BEDOK, CHOA CHU KANG, YISHUN)",0.061728,0.257202,0.061728,1.000000,3.888,0.045852,inf
...,...,...,...,...,...,...,...,...,...
7737528,(YISHUN),"(JURONG WEST, JURONG EAST, PUNGGOL, CHOA CHU K...",1.000000,0.061728,0.061728,0.061728,1.000,0.000000,1.0
7737530,(SEMBAWANG),"(JURONG WEST, JURONG EAST, PUNGGOL, CHOA CHU K...",1.000000,0.061728,0.061728,0.061728,1.000,0.000000,1.0
7741609,"(SEMBAWANG, YISHUN)","(JURONG WEST, JURONG EAST, PUNGGOL, CHOA CHU K...",1.000000,0.061728,0.061728,0.061728,1.000,0.000000,1.0
7741622,(YISHUN),"(JURONG WEST, JURONG EAST, PUNGGOL, CHOA CHU K...",1.000000,0.061728,0.061728,0.061728,1.000,0.000000,1.0
